In [ ]:
#r "nuget: Graphoscope, 0.5.1-preview.1"


# What is AdjGraph

The AdjGraph is an undirected adaptation of the FGraph.
It combines the predecessors and successors into just one element called neighbours.

[![](https://mermaid.ink/img/pako:eNpdjssOgjAQRX-FzAoT-AEWJgjIjo3urIuxHR7yKCltDCH8u7VijM5qcs7NzF2AS0EQQdnJB69Rae-cssGzE19icc8VjvXVC8O9d_ALG21p3m3e0cRRm-Q08I9KnEqdSlHjD878gpqqvkmjpk1kThz_Hrxp7mei-l6BAHpSPTbCll5ehIGuqScGkV0FqpYBG1abQ6PlaR44RFoZCsCMAjWlDVYKe4hK7CZan7YATRE?type=png)](https://mermaid.live/edit#pako:eNpdjssOgjAQRX-FzAoT-AEWJgjIjo3urIuxHR7yKCltDCH8u7VijM5qcs7NzF2AS0EQQdnJB69Rae-cssGzE19icc8VjvXVC8O9d_ALG21p3m3e0cRRm-Q08I9KnEqdSlHjD878gpqqvkmjpk1kThz_Hrxp7mei-l6BAHpSPTbCll5ehIGuqScGkV0FqpYBG1abQ6PlaR44RFoZCsCMAjWlDVYKe4hK7CZan7YATRE)

# Quickstart

## Creating an empty graph and filling it with single elements

Begin by creating an empty graph,meaning a data structure with no nodes or edges.
Then populate the graph with single elements, individual nodes are added one by one, and edges can be introduced to establish connections between them.



In [3]:
open Graphoscope
open AdjGraph

let graphToFillAdjGraph =

    AdjGraph.empty
    |> AdjGraph.addNode 1 "1"
    |> AdjGraph.addNode 2 "2"
    |> AdjGraph.addEdge 1 2 1.


"You have created a graph with 2 nodes and 1 edges"

# Working with Graphs

## Creating a Graph using AdjGraph

### Creating an empty graph and add collections of elements

Another way of creating a graph is by filling it with collections of nodes and edges as seen below:



In [6]:
let graphToFillAdjGraph' =
    
    let nodes = List.init 100 (fun x -> x,$"{x}")

    let edges = List.init 45 (fun x -> x,x*2,1.)

    AdjGraph.empty
    |> AdjGraph.addNodes nodes
    |> AdjGraph.addEdges edges


"You have created a graph with 100 nodes and 45 edges"

### Removing Nodes and Edges

To remove Nodes or Edges you can just use the remove functions provided:



In [9]:
let graphWithRemovedElementsAdjGraph =
    graphToFillAdjGraph'
    |> AdjGraph.removeEdge 1 2
    |> AdjGraph.removeNode 0


"You have reduced the graph to 99 nodes and 43 edges"

## 

# From Data

## Import a graph

This is the well-known and much-used Zachary karate club network. The data was collected from the members of a university karate club by Wayne Zachary in 1977.
Each node represents a member of the club, and each edge represents a tie between two members of the club.
The network is undirected.



In [12]:
open FSharp.Data

let getElementOfFile (fullpath: string) (delimiter: string) (headerRows: int) (weightsIncluded: bool)  = 
        let rows  = CsvFile.Load(fullpath, delimiter, skipRows = headerRows, hasHeaders = false).Rows
        rows
        |> Seq.map (fun row -> int row[0],int row[0], int row[1],int row[1], if weightsIncluded then float row[2] else 1.0)


let karateFileAdj= __SOURCE_DIRECTORY__ + "/../tests/Graphoscope.Tests/ReferenceGraphs/zachary.txt"

let karateGraphAdj = 
  let g = AdjGraph.empty<int,int,float>
  getElementOfFile karateFileAdj " " 2 false
  |> Seq.iter(fun (s1,s2,t1,t2,w: float) -> AdjGraph.addElement s1 s2 t1 t2 w g|>ignore)
  g


let's use Cytoscape.NET for visualization:



In [13]:
open Cytoscape.NET
let vizGraph =
    CyGraph.initEmpty ()
    |> CyGraph.withElements [
            for (sk,s,tk,t,el) in (AdjGraph.toSeq karateGraphAdj) do
                let sk, tk = (string sk), (string tk)
                yield Elements.node sk [ CyParam.label s ]
                yield Elements.node tk [ CyParam.label t ]
                yield Elements.edge  (sprintf "%s_%s" sk tk) sk tk [ CyParam.label el ]
        ]
    |> CyGraph.withStyle "node"     
        [
            CyParam.content =. CyParam.label
            CyParam.color "#A00975"
        ]
    |> CyGraph.withLayout(Cytoscape.NET.Layout.initBreadthfirst(id))


<!-- Cytoscape graph will be drawn inside this DIV -->